<h1> COMPETICIÓN KAGGLE <h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Leer el archivo CSV
df_train = pd.read_csv('train.csv')

In [4]:
df_train.isnull().sum()

id                      0
date                    0
is_weekend              0
steps_total         32011
distance            49951
sleep               52770
app_usage           66154
home_cluster        98564
clusters_count      91760
t_sport             57669
practiced_sport     57669
valence            100994
dtype: int64

Donde más missing values hay es en la valencia

In [ ]:
df_train = df_train.dropna(subset=['valence'])

In [ ]:
df_train.isnull().sum()

id                    0
date                  0
is_weekend            0
steps_total        1571
distance           4671
sleep              1578
app_usage          3653
home_cluster       8299
clusters_count     7038
t_sport            2121
practiced_sport    2121
valence               0
dtype: int64

In [ ]:
df_train.shape

(9703, 12)

## Procesado datos 1, con la media para cada paciente en cada columna, pero para los pasos se hace un promer bucle con una ventana de 5 dias

In [4]:
def mode_if_exists(x):
    if not x.dropna().empty:
        return x.mode().iloc[0]
    else:
        return pd.NA

In [5]:
def procesado_datos(df):
    # Primero, ordena el DataFrame por la columna de fecha
    df.sort_values(by=['id', 'date'], inplace=True)

    # Creamos un bucle para iterar sobre cada grupo de pacientes
    for paciente, datos_paciente in df.groupby('id'):
        # Calculamos la media móvil para el paciente actual
        rolling_mean = datos_paciente['steps_total'].rolling(window=5, min_periods=1).mean()
        # Imputamos los valores faltantes para el paciente actual
        df.loc[df['id'] == paciente, 'steps_total'] = df.loc[df['id'] == paciente, 'steps_total'].fillna(rolling_mean)

    # Calculamos la media de los pasos totales para cada paciente
    mean_steps = df.groupby('id')['steps_total'].transform('mean')
    # Llenamos los valores faltantes para cada paciente con la media respectiva
    df['steps_total'].fillna(mean_steps, inplace=True)

    mean_sleep = df.groupby('id')['sleep'].transform('mean')
    df['sleep'].fillna(mean_sleep, inplace=True)

    # Calculamos la media de la distancia para cada paciente
    mean_distance = df.groupby('id')['distance'].transform('mean')
    # Llenamos los valores faltantes para cada paciente con la media respectiva
    df['distance'].fillna(mean_distance, inplace=True)

    mean_home = df.groupby('id')['home_cluster'].transform('mean')
    # Llenamos los valores faltantes para cada paciente con la media respectiva
    df['home_cluster'].fillna(mean_home, inplace=True)

    # Calculamos la media de los clusters para cada paciente
    mean_cluster = df.groupby('id')['clusters_count'].transform('mean')
    # Llenamos los valores faltantes para cada paciente con la media respectiva
    df['clusters_count'].fillna(mean_cluster, inplace=True)

    # Calculamos la media de la t_sport para cada paciente
    mean_sport = df.groupby('id')['t_sport'].transform('mean')
    # Llenamos los valores faltantes para cada paciente con la media respectiva
    df['t_sport'].fillna(mean_sport, inplace=True)

    # Calcular la moda por grupo 'id'
    mode_practiced = df.groupby('id')['practiced_sport'].transform(mode_if_exists)
    # Imputar los valores faltantes con la moda calculada
    df['practiced_sport'].fillna(mode_practiced)

    # Calculamos la media de cada columna
    mean_steps_total = df['steps_total'].mean()
    mean_distance = df['distance'].mean()
    mean_sleep = df['sleep'].mean()
    mean_app_usage = df['app_usage'].mean()
    mean_home_cluster = df['home_cluster'].mean()
    mean_clusters_count = df['clusters_count'].mean()
    mean_t_sport = df['t_sport'].mean()
    mode_practiced_sport = df['practiced_sport'].mode().iloc[0]
    # Llenamos los valores faltantes con la media de cada columna
    df['steps_total'].fillna(mean_steps_total, inplace=True)
    df['distance'].fillna(mean_distance, inplace=True)
    df['sleep'].fillna(mean_sleep, inplace=True)
    df['app_usage'].fillna(mean_app_usage, inplace=True)
    df['home_cluster'].fillna(mean_home_cluster, inplace=True)
    df['clusters_count'].fillna(mean_clusters_count, inplace=True)
    df['t_sport'].fillna(mean_t_sport, inplace=True)
    df['practiced_sport'].fillna(mode_practiced_sport, inplace=True)


In [6]:
procesado_datos(df_train)

In [7]:
df_train.isnull().sum()

id                      0
date                    0
is_weekend              0
steps_total             0
distance                0
sleep                   0
app_usage               0
home_cluster            0
clusters_count          0
t_sport                 0
practiced_sport         0
valence            100994
dtype: int64

## Creación nuevo conjunto de datos

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
new_df = []
new_df_y = []
df_train['date'] = pd.to_datetime(df_train['date'])  # Convertir la columna 'date' a tipo Timestamp
features = ['id','is_weekend', 'steps_total', 'distance', 'sleep', 'app_usage', 'home_cluster', 'clusters_count', 't_sport', 'practiced_sport']
for index, row in df_train.iterrows():
    if pd.isna(row['valence']):
      pass
    else:
      fecha_hoy = row['date']
      id_paciente = row['id']
      #print('fecha_hoy: ',fecha_hoy)
      #fecha_inicio = fecha_hoy - pd.Timedelta(days=7)
      # print('fecha_inicio: ',fecha_inicio)
      # index_fechas = pd.date_range(start=fecha_inicio, end=fecha_hoy, freq='D')
      index_num = np.arange(0,7,1)
      #print(index)
      if(index > 6):
        #print('fecha_inicio: ',fecha_inicio)
        #print('indece: ',index_fechas)
        X_new = pd.DataFrame(index = index_num,columns=features).fillna(0)
        #print(df_train.loc[index][features].values)
        X_new.loc[7] = df_train.loc[index][features].values
        #print(X_new)
        for i in range(1,8,1):
          #print('index-i: ',index-i)
          # print(df_train.loc[index-i])
          #print('id_bucle: ',df_train.loc[index-i]['id'])
          #print('fecha: ', index_fechas[index-i])
          #print('fila:  ',X_new.loc[index_fechas[index-i]])
          if (df_train.loc[index-i]['id']==id_paciente):
            X_new.loc[7-i] = (df_train.loc[index-i][features].values)
            #print(X_new)
          else:
            X_new.loc[7-i] = (np.zeros(len(features)))
            #print(X_new)
        #print('fecha_hoy: ',fecha_hoy)
        #print('index: ',index)
        #print(X_new)
        #print('--------------------')
        #print(X_new)
        '''
        X_new['date'] = pd.to_datetime(X_new['date'])
        X_new = X_new.sort_values('date')

        # Reindexa X_new basado en 'date' columna y reemplaza cualquier valor NaN con el valor anterior
        X_new = X_new.set_index('date').reindex(pd.date_range(start=X_new['date'].min(), end=X_new['date'].max(), freq='D')).ffill()
        #print('reindex--------------------')
        '''
        #print(X_new)
        #y_new = row['valence']
        new_df.append(X_new)
        new_df_y.append(row['valence'])


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf

def new_data(df):
  new_df_x = []
  new_df_y = []
  df['date'] = pd.to_datetime(df['date'])  # Convertir la columna 'date' a tipo Timestamp
  features = ['id','is_weekend', 'steps_total', 'distance', 'sleep', 'app_usage', 'home_cluster', 'clusters_count', 't_sport', 'practiced_sport']
  for index, row in df.iterrows():
      if pd.isna(row['valence']):
        pass
      else:
        fecha_hoy = row['date']
        id_paciente = row['id']
        index_num = np.arange(0,7,1)
        if(index > 6):
          X_new = []
          X_new.insert(0,df.loc[index][features].values)
          for i in range(1,8,1):
            if (df.loc[index-i]['id']==id_paciente):
              X_new.insert(0,(df.loc[index-i][features].values))
            else:
              X_new.insert(0,(np.zeros(len(features))))
          new_df_x.append(X_new)
          new_df_y.append(row['valence'])
  return new_df_x, new_df_y

In [9]:
X_train_new, y_train_new = new_data(df_train)

In [10]:
x_train_array = np.array(X_train_new)

In [11]:
x_train_array.shape

(9702, 8, 10)

In [ ]:
x_train_array[0]

array([[5, 0, -0.6064562300110155, -0.35049734610679684,
        -0.3299789587223714, -0.161625313002697, -0.3026979329475962,
        -0.44100151427013184, -0.6868633739853445, 1.0],
       [5, 1, -0.6064562300110155, -0.35049734610679684,
        -0.3299789587223714, 1.992984979946306, -0.3026979329475962,
        -0.44100151427013184, -0.6868633739853445, 1.0],
       [5, 1, -0.6064562300110155, -0.35049734610679684,
        -0.3299789587223714, 1.0384688545529142, -0.3026979329475962,
        -0.44100151427013184, -0.6868633739853445, 1.0],
       [5, 0, -0.6064562300110155, -0.35049734610679684,
        -0.3299789587223714, 1.1994079450021786, -0.3026979329475962,
        -0.44100151427013184, -0.6868633739853445, 1.0],
       [5, 0, -0.6064562300110155, -0.35049734610679684,
        -0.3299789587223714, 2.185055638841716, -0.3026979329475962,
        -0.44100151427013184, -0.6868633739853445, 1.0],
       [5, 0, -0.6064562300110155, -0.35049734610679684,
        -0.32997895872237

In [12]:
y_train_new_df = pd.DataFrame (y_train_new)
y_train_array = y_train_new_df.values
y_train_array.shape

(9702, 1)

## Entrenamiento de la lstm

In [20]:
# Dividir los datos en conjuntos de entrenamiento y prueba
# Escalado de características
X_train, X_val, y_train, y_val = train_test_split(x_train_array, y_train_array, test_size=0.2, random_state=42)

# Definir el modelo LSTM
model = Sequential([
    LSTM(units=64, input_shape=(x_train_array.shape[1], x_train_array.shape[2])),  # 7 días de datos, 11 características
    Dense(units=3),  # Una neurona de salida para la predicción de valencia
    Activation('softmax')
])

x_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Entrenar el modelo
model.fit(x_train_tensor, y_train_tensor, validation_data=(X_val_tensor, y_val_tensor), epochs=10, batch_size=32)

# Evaluar el modelo en los datos de validación
loss = model.evaluate(X_val_tensor, y_val_tensor)
print("Loss en datos de validación:", loss)

Epoch 1/10
243/243 [==============================] - 6s 14ms/step - loss: 2.1837 - val_loss: 1.0986
Epoch 2/10
243/243 [==============================] - 2s 8ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 3/10
243/243 [==============================] - 2s 8ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 4/10
243/243 [==============================] - 2s 7ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 5/10
243/243 [==============================] - 2s 7ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 6/10
243/243 [==============================] - 2s 7ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 7/10
243/243 [==============================] - 2s 8ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 8/10
243/243 [==============================] - 3s 11ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 9/10
243/243 [==============================] - 2s 8ms/step - loss: 1.0986 - val_loss: 1.0986
Epoch 10/10
61/61 [==============================] - 0s 4ms/step - loss: 1.0986
Loss en datos de v

## Crear el csv con las predicciones a partir del csv de test

In [21]:
# Read the test.csv file
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/test.csv')

In [22]:
print(test_df.isnull().sum())

id                     0
date                   0
is_weekend             0
steps_total         6063
distance           11283
sleep              10576
app_usage          14521
home_cluster       22711
clusters_count     21445
t_sport            11422
practiced_sport    11422
dtype: int64


In [23]:
procesado_datos(test_df)
print(test_df.isnull().sum())

id                 0
date               0
is_weekend         0
steps_total        0
distance           0
sleep              0
app_usage          0
home_cluster       0
clusters_count     0
t_sport            0
practiced_sport    0
dtype: int64


In [37]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf

def new_data_test(df):
  new_df_x = []
  features = ['id','is_weekend', 'steps_total', 'distance', 'sleep', 'app_usage', 'home_cluster', 'clusters_count', 't_sport', 'practiced_sport']
  for index, row in df.iterrows():
        id_paciente = row['id']
        index_num = np.arange(0,7,1)
        if(index > 6):
          X_new = []
          X_new.insert(0,df.loc[index][features].values)
          for i in range(1,8,1):
            if (df.loc[index-i]['id']==id_paciente):
              X_new.insert(0,(df.loc[index-i][features].values))
            else:
              X_new.insert(0,(np.zeros(len(features))))
          new_df_x.append(X_new)
        else:
          X_new = []
          X_new.insert(0,df.loc[index][features].values)
          for i in range(1,8,1):
              X_new.insert(0,(np.zeros(len(features))))
          new_df_x.append(X_new)
  return new_df_x

In [38]:
X_test_new = new_data_test(test_df)
x_test_array = np.array(X_test_new)
x_test_array.shape
x_test_tensor = tf.convert_to_tensor(x_test_array, dtype=tf.float32)

In [42]:
# test_df.drop('date',axis=1, inplace = True)
# Hacer predicciones sobre el conjunto de prueba para todo el rango de fechas
test_predictions = model.predict(x_test_tensor)


829/829 [==============================] - 3s 3ms/step


In [39]:

# Print the predictions
# Read the initial test.csv file
initial_test_df = pd.read_csv('/content/drive/MyDrive/Kaggle/test.csv')

# Add the valence feature with the results
initial_test_df['valence'] = test_predictions

# Load the test_prediction.csv file
test_prediction_df = pd.read_csv('/content/drive/MyDrive/Kaggle/test_prediction.csv')

# Merge the test_prediction_df with initial_test_df on 'id' and 'date'
# Merge the initial_test_df with test_prediction_df on 'id' and 'date'
# Realizar el merge
df_merged = pd.merge(test_prediction_df, initial_test_df[['valence', 'date', 'id']], on=['date', 'id'], how='left')


# Display the updated dataframe
print(df_merged)

829/829 [==============================] - 2s 3ms/step


ValueError: Expected a 1D array, got an array with shape (26516, 3)

In [ ]:
# Paso 1: Renombrar las columnas
df_merged.rename(columns={'id_kaggle': 'Id', 'valence': 'Category'}, inplace=True)

# Paso 2: Convertir y ajustar la columna 'Category'
# Convertir 'Category' a entero para eliminar cualquier decimal, luego a string
df_merged['Category'] = df_merged['Category'].astype(int).astype(str)
# Añadir comillas simples alrededor de los valores de 'Category'
df_merged['Category'] = "'" + df_merged['Category'] + "'"

# Paso 3: Seleccionar solo las columnas de interés (Id y Category)
df_final = df_merged[['Id', 'Category']]

# Paso 4: Guardar el DataFrame en un archivo CSV
df_final.to_csv('/content/drive/MyDrive/Kaggle/output_10_interpolate_rf_gs.csv', index=False)
